In [ ]:
import numpy as np
import scipy
import pandas as pd

In [ ]:
URL = "http://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data"
df = pd.read_csv(URL)


In [ ]:
Attribute = ["символизация",
             "нормализованные потери",
             "марка",
             "вид топлива",
             "стремление",
             "количество дверей",
             "тип кузова",
             "ведущий мост",
             "расположение двигателя",
             "колесная база",
             "длина",
             "ширина",
             "высота",
             "снаряженная масса",
             "тип двигателя",
             "количество цилиндров",
             "объем двигателя",
             "топливная система",
             "диаметр отверстия",
             "ход",
             "степень сжатия",
             "мощность двигателя",
             "пиковые обороты",
             "городской режим",
             "шоссейный режим",
             "цена"
            ]

In [ ]:
df = pd.read_csv(URL,names = Attribute)

In [ ]:
missed = (df == '?').sum()
percentage = missed / df.shape[0]
nan_cols=percentage[percentage != 0].index
percentage = percentage[percentage != 0]

In [ ]:
df1 = df.copy(deep = True)
col_nan_without_norm_less = nan_cols[1:]
col_nan_without_norm_less
df1 = df.copy(deep = True)
for c in col_nan_without_norm_less:
    df1.loc[df1[c] == '?',c] = np.nan
df1 = df1.dropna()
del_pers = df1.shape[0]/df.shape[0]
df.loc[df['нормализованные потери'] == '?','нормализованные потери'] = np.nan
df['нормализованные потери'] = df['нормализованные потери'].astype(float)

In [ ]:
df1.loc[df1['нормализованные потери'] == '?','нормализованные потери'] = np.nan
df1 = df1.fillna(df1['нормализованные потери'].median())
df1['нормализованные потери'] = df1['нормализованные потери'].astype(float)
df1['цена'] = df1['цена'].astype(float)

In [ ]:
import scipy.stats as stats
# Метод доверительных интервалов.
def trust_intervals(data = df):
    df1 = data.copy(deep = True)
    not_object_col = [c for c in df1.columns if df1[c].dtype != 'object']
    # Задаем уровень доверия - 95 (обычно 95)
    confidence = 0.999
    for c in not_object_col:
        mean = data[c].mean()
        std_dev = data[c].std()
        # Вычисляем доверительный интервал
        interval = stats.norm.interval(confidence,loc = mean,scale = std_dev)
        # Исключаем из выборки данные, выходящие за интервал
        # interval[1] - верхняя граница, interval[0] - нижняя граница интервала.
        df1 = df1[(df1[c] >= interval[0])&(df1[c] <= interval[1])]
    return df1

In [ ]:
df2 = trust_intervals(df1).copy(deep = True)

In [ ]:
from scipy.stats import mannwhitneyu
groupA, groupB = np.array(df2['городской режим']), np.array(df2['шоссейный режим'])
u_stat, p_value = mannwhitneyu(groupA, groupB, method="exact")
print('U-statistic:', u_stat)
print('P-value:', p_value)